## Train a Scikit-Learn Model using SageMaker Container Mode
#### Bring Your Own Container (BYOC)

### 1. Create Train Script 

In [1]:
%%file train
#!/usr/bin/env python

from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import pickle
import os


np.random.seed(123)

# Define paths for Model Training inside Container.
INPUT_PATH = '/opt/ml/input/data'
OUTPUT_PATH = '/opt/ml/output'
MODEL_PATH = '/opt/ml/model'
PARAM_PATH = '/opt/ml/input/config/hyperparameters.json'

# Training data sitting in S3 will be copied to this location during training when used with File MODE.
TRAIN_DATA_PATH = f'{INPUT_PATH}/train'
TEST_DATA_PATH = f'{INPUT_PATH}/test'

def train():
    print("------- [STARTING TRAINING] -------")
    train_df = pd.read_csv(os.path.join(TRAIN_DATA_PATH, 'train.csv'), names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
    train_df.head()
    X_train = train_df[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']]
    y_train = train_df['class']
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    # Save the trained Model inside the Container
    with open(os.path.join(MODEL_PATH, 'model.pkl'), 'wb') as out:
        pickle.dump(knn, out)
    print("------- [TRAINING COMPLETE!] -------")
    
    print("------- [STARTING EVALUATION] -------")
    test_df = pd.read_csv(os.path.join(TEST_DATA_PATH, 'test.csv'), names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])
    X_test = train_df[['bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate']]
    y_test = train_df['class']
    acc = knn.score(X_test, y_test)
    print('Accuracy = {:.2f}%'.format(acc * 100))
    print("------- [EVALUATION DONE!] -------")

if __name__ == '__main__':
    train()

Overwriting train


### 2. Create Serve Script

In [2]:
%%file serve
#!/usr/bin/env python

from flask import Flask, Response, request
from io import StringIO
import pandas as pd
import logging
import pickle
import os


app = Flask(__name__)

MODEL_PATH = '/opt/ml/model'

# Singleton Class for holding the Model
class Predictor:
    model = None
    
    @classmethod
    def load_model(cls):
        print('[LOADING MODEL]')
        if cls.model is None:
            with open(os.path.join(MODEL_PATH, 'model.pkl'), 'rb') as file_:
                cls.model = pickle.load(file_)
        print('MODEL LOADED!')
        return cls.model
    
    @classmethod
    def predict(cls, X):
        clf = cls.load_model()
        return clf.predict(X)

@app.route('/ping', methods=['GET'])
def ping():
    print('[HEALTH CHECK]')
    model = Predictor.load_model()
    status = 200
    if model is None:
        status = 404
    return Response(response={"HEALTH CHECK": "OK"}, status=status, mimetype='application/json')

@app.route('/invocations', methods=['POST'])
def invoke():
    data = None

    # Transform Payload in CSV to Pandas DataFrame.
    if request.content_type == 'text/csv':
        data = request.data.decode('utf-8')
        data = StringIO(data)
        data = pd.read_csv(data, header=None)
    else:
        return flask.Response(response='This Predictor only supports CSV data', status=415, mimetype='text/plain')

    logging.info('Invoked with {} records'.format(data.shape[0]))
    
    predictions = Predictor.predict(data)

    # Convert from numpy back to CSV
    out = StringIO()
    pd.DataFrame({'results': predictions}).to_csv(out, header=False, index=False)
    result = out.getvalue()

    return Response(response=result, status=200, mimetype='text/csv')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

Overwriting serve


### 3. Build a Docker Image and Push to ECR

<p>Build the docker image and push to ECR and have the image URI handy for the next steps.</p>

In [3]:
!docker build -t sagemaker-byoc-sklearn -f Dockerfile .

Sending build context to Docker daemon  41.47kB
Step 1/8 : FROM python:3.7
3.7: Pulling from library/python

565cc8df: Pulling fs layer 
d13e55e7: Pulling fs layer 
7528c685: Pulling fs layer 
072f9cd1: Pulling fs layer 
83117533: Pulling fs layer 
2d56ded5: Pulling fs layer 
f01be008: Pulling fs layer 
fb36b5a8: Pulling fs layer 
Digest: sha256:8b743b1af852e554b98e2377f9c92221693225b85d984b23be6b033018f97cc3[6A
Status: Downloaded newer image for python:3.7
 ---> f97db05f776a
Step 2/8 : COPY requirements.txt ./
 ---> b9c805c17af7
Step 3/8 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Running in 9211409e654f
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 204.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 186.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 KB 234.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 204.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.

In [4]:
%%sh

# Specify a name to your custom container
container_name=sagemaker-byoc-sklearn
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

Container Name:  sagemaker-byoc-sklearn
Account:  119174016168
Region: us-east-1
ECR Repository:  119174016168.dkr.ecr.us-east-1.amazonaws.com
ECR Image URI:  119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker-byoc-sklearn:latest
Login Succeeded
The push refers to repository [119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker-byoc-sklearn]
575b202a40db: Preparing
ee795eea6e0c: Preparing
9b7e8570903d: Preparing
19290b8c0c93: Preparing
de4fe2acc881: Preparing
565fe7d7f6e2: Preparing
bd2aedacc58f: Preparing
0d858d352b03: Preparing
76bc459f764a: Preparing
100796cdf3b1: Preparing
54acb5a6fa0b: Preparing
8d51c618126f: Preparing
9ff6e4d46744: Preparing
a89d1d47b5a1: Preparing
655ed1b7a428: Preparing
bd2aedacc58f: Waiting
54acb5a6fa0b: Waiting
565fe7d7f6e2: Waiting
8d51c618126f: Waiting
655ed1b7a428: Waiting
9ff6e4d46744: Waiting
76bc459f764a: Waiting
a89d1d47b5a1: Waiting
100796cdf3b1: Waiting
0d858d352b03: Waiting
575b202a40db: Pushed
9b7e8570903d: Pushed
ee795eea6e0c: Pushed
19290b8

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### 4. Train your Custom Sklearn Model using SageMaker Training

### Imports 

In [5]:
from sagemaker.serializers import CSVSerializer
import pandas as pd
import sagemaker

### Essentials

In [15]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
image_name = 'sagemaker-byoc-sklearn'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'

In [16]:
image_uri

'119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker-byoc-sklearn:latest'

### Train (using SageMaker)

In [17]:
WORK_DIRECTORY = '.././DATA'

train_data_s3_pointer = session.upload_data(f'{WORK_DIRECTORY}/train', key_prefix='byoc-sklearn/train')
test_data_s3_pointer = session.upload_data(f'{WORK_DIRECTORY}/test', key_prefix='byoc-sklearn/test')

In [18]:
train_data_s3_pointer

's3://sagemaker-us-east-1-119174016168/byoc-sklearn/train'

In [19]:
test_data_s3_pointer

's3://sagemaker-us-east-1-119174016168/byoc-sklearn/test'

In [20]:
model = sagemaker.estimator.Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    sagemaker_session=session  # ensure the session is set to session
)

In [21]:
model.fit({'train': train_data_s3_pointer, 'test': test_data_s3_pointer})

2022-08-24 15:03:56 Starting - Starting the training job...
2022-08-24 15:04:19 Starting - Preparing the instances for trainingProfilerReport-1661353436: InProgress
......
2022-08-24 15:05:20 Downloading - Downloading input data...
2022-08-24 15:05:43 Training - Downloading the training image...
2022-08-24 15:06:25 Uploading - Uploading generated training model
2022-08-24 15:06:25 Completed - Training job completed
------- [STARTING TRAINING] -------
------- [TRAINING COMPLETE!] -------
------- [STARTING EVALUATION] -------
Accuracy = 82.42%
------- [EVALUATION DONE!] -------
Training seconds: 67
Billable seconds: 67


In [25]:
model._current_job_name

'sagemaker-byoc-sklearn-2022-08-24-15-03-56-118'

### Create Endpoint Config

In [52]:
import datetime
from time import gmtime, strftime

In [53]:
#TRAINING_JOB_NAME = 'classifier-2020-12-07-19-55-02-397' # Copy this from the AWS SageMaker console
TRAINING_JOB_NAME = model._current_job_name

In [54]:
sagemaker_session = sagemaker.session.Session()

In [55]:
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
MODEL_NAME = f'clf-xgboost-model-{current_timestamp}'

In [56]:
model_name = sagemaker_session.create_model_from_job(training_job_name=TRAINING_JOB_NAME, 
                                        name=MODEL_NAME)

In [57]:
s3_bucket = sagemaker_session.default_bucket()
bucket_prefix = 'async_test'

In [65]:
s3_bucket

'sagemaker-us-east-1-119174016168'

In [58]:
import boto3
# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')

In [62]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = f"async-ep-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
print(endpoint_config_name)


create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": "ml.m5.xlarge", # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            # Location to upload response outputs when no location is provided in the request.
            "S3OutputPath": f"s3://{s3_bucket}/{bucket_prefix}/output",
            # (Optional) specify Amazon SNS topics
            "NotificationConfig": {
                "SuccessTopic": "arn:aws:sns:us-east-1:119174016168:success-topic",
                "ErrorTopic": "arn:aws:sns:us-east-1:119174016168:error-topic",
            }
        },
        "ClientConfig": {
            # (Optional) Specify the max number of inflight invocations per instance
            # If no value is provided, Amazon SageMaker will choose an optimal value for you
            "MaxConcurrentInvocationsPerInstance": 4
        }
    }
)

#print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

async-ep-2022-08-24-15-48-50


### Create Async Endpoint

In [63]:
# The name of the endpoint.The name must be unique within an AWS Region in your AWS account.
endpoint_name = f"async-ep-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}" 

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [64]:
endpoint_name

'async-ep-2022-08-24-15-49-04'

### Invoke Async Endpoint

In [66]:
# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

# Specify the location of the input. Here, a single SVM sample
input_location = f"s3://{s3_bucket}/async-test/test.csv"


# After you deploy a model into production using SageMaker hosting 
# services, your client applications use this API to get inferences 
# from the model hosted at the specified endpoint.
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)
response

{'ResponseMetadata': {'RequestId': '38ab7a22-4aa1-4d70-86a3-135990257ba9',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': '38ab7a22-4aa1-4d70-86a3-135990257ba9',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/07294395-3950-46e8-be75-443d4248133c.out',
   'date': 'Wed, 24 Aug 2022 15:56:32 GMT',
   'content-type': 'application/json',
   'content-length': '54'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/07294395-3950-46e8-be75-443d4248133c.out',
 'InferenceId': 'de69aef3-45b9-42d5-b5b2-43129b60e3a4'}

### Invoke Async Endpoint (Exception Scenario)

In [67]:
# Create a low-level client representing Amazon SageMaker Runtime
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')

# Specify the location of the input. Here, a single SVM sample
input_location = f"s3://{s3_bucket}/async-test/bad_test.csv"  # 5 col value is string


# After you deploy a model into production using SageMaker hosting 
# services, your client applications use this API to get inferences 
# from the model hosted at the specified endpoint.
response = sagemaker_runtime.invoke_endpoint_async(
                            EndpointName=endpoint_name, 
                            InputLocation=input_location)
response

{'ResponseMetadata': {'RequestId': '2814a109-75b3-4281-b89c-96bc30931970',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': '2814a109-75b3-4281-b89c-96bc30931970',
   'x-amzn-sagemaker-outputlocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/7c7af8ff-bd8a-4426-b9dd-af6e439ebfd0.out',
   'date': 'Wed, 24 Aug 2022 16:10:46 GMT',
   'content-type': 'application/json',
   'content-length': '54'},
  'RetryAttempts': 0},
 'OutputLocation': 's3://sagemaker-us-east-1-119174016168/async_test/output/7c7af8ff-bd8a-4426-b9dd-af6e439ebfd0.out',
 'InferenceId': '6236a879-e78a-44b6-bc98-21ba5d744ad4'}

In [22]:
#Ignore everything below for now.

### Deploy Trained Model as SageMaker Endpoint

In [10]:
csv_serializer = CSVSerializer()
predictor = model.deploy(1, 'ml.m5.xlarge', 
                         endpoint_name='emr-byoc-sklearn', 
                         serializer=csv_serializer)

-------------!

### Real Time Inference using Deployed Endpoint

In [11]:
df = pd.read_csv('.././DATA/test/test.csv', header=None)
test_df = df.sample(1)

In [12]:
test_df.drop(test_df.columns[[0]], axis=1, inplace=True)
test_df

,1,2,3,4
1608,0.733637,0.347981,0.228029,0.162324


In [13]:
test_df.values

array([[0.73363737, 0.3479813 , 0.22802851, 0.16232361]])

In [14]:
prediction = predictor.predict(test_df.values).decode('utf-8').strip()

In [15]:
prediction

'0'